In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_empirical_variable_cost_runs(acq, tfn):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "adedfoxa", 
        "config.problem": tfn,
        "config.policy": acq,
        "config.cost_function_type": "unknown"})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
    'RandomSearch': 'RandomSearch',
    'ExpectedImprovementWithoutCost':'ExpectedImprovementWithoutCost',
    'ExpectedImprovementPerUnitCost':'ExpectedImprovementPerUnitCost',
    'BudgetedMultiStepLookaheadEI': 'BudgetedMultiStepLookaheadEI',
    # 'Gittins_Lambda_0001':'Gittins_Lambda0001',
    # 'Gittins_Step_Divide2':'Gittins_Step_Divide2',
    }
target_functions = ["RobotPushing14D"]

In [3]:
grouped_runs = {(a,t): load_empirical_variable_cost_runs(a,t) for a in acquisition_functions.keys() for t in target_functions}

100%|██████████| 15/15 [00:10<00:00,  1.38it/s]


In [7]:
for a in acquisition_functions.keys():
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T

        # Handling potential empty arrays
        if cumulative_cost_per_seed.size == 0 or best_observed_per_seed.size == 0:
            continue  # Skip this iteration if there's no data

        print(a, t, best_observed_per_seed.shape)
        
        best_25 = np.quantile(best_observed_per_seed, 0.25, axis=1)
        best_50 = np.quantile(best_observed_per_seed, 0.5, axis=1)
        best_75 = np.quantile(best_observed_per_seed, 0.75, axis=1)

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), best_25, best_50, best_75),axis=-1)

        np.savetxt(f"results/quartiles/empirical_cost_aware/Empirical_VariableCost_{t}_unknown_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

RandomSearch RobotPushing14D (801, 16)
ExpectedImprovementWithoutCost RobotPushing14D (801, 16)
ExpectedImprovementPerUnitCost RobotPushing14D (801, 16)
Gittins_Lambda_0001 RobotPushing14D (801, 15)
Gittins_Step_Divide2 RobotPushing14D (801, 15)


In [4]:
for a in acquisition_functions.keys():
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0]).T 
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['best observed'])>0]).T
            
        print(a, t, best_observed_per_seed.shape)

        mean = np.mean(best_observed_per_seed, axis=1)
        se = np.std(best_observed_per_seed, axis=1, ddof=1) / np.sqrt(best_observed_per_seed.shape[1])
        
        mean_plus_2se = mean + 2 * se
        mean_minus_2se = mean - 2 * se

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), mean_minus_2se, mean, mean_plus_2se),axis=-1)

        np.savetxt(f"results/SE/empirical_cost_aware/Empirical_VariableCost_{t}_unknown_{acquisition_functions[a]}.csv", output, header="cc, mean-2se, mean, mean+2se", delimiter=', ', comments='')

RandomSearch RobotPushing14D (801, 16)
ExpectedImprovementWithoutCost RobotPushing14D (801, 16)
ExpectedImprovementPerUnitCost RobotPushing14D (801, 16)
BudgetedMultiStepLookaheadEI RobotPushing14D (801, 8)
